### HyperLogLog v4

In [3]:
sc.stop()

In [7]:
# download_file 
#! beeline -u "jdbc:hive2://ds-hadoop-cs01p:10000/" -n kposminin --incremental=true --showheader=false --outputformat=tsv2 
# --maxwidth=5000 --silent=true --showWarnings=false --showNestedErrs=false --verbose=false --nullemptystring=true 
#-e "select phone_mobile, approve, call_ymd, urlfr, ymd from user_kposminin.ccall_visits order by phone_mobile" > ccall_cred_visits.txt
#! head -1 /data1/share/kosm/data/ccall_cred_visits.txt


In [8]:
from HLL import HyperLogLog as Hll

hll_len = 14

salts = ['32m4','sd8f','9zj1']
i = 0
prev_phone_mobile = None
prev_approve = None
buf = {} #Hll(hll_len)
with open('/data1/share/kosm/data/ccall_cred_visits.txt','r') as f:
    with open('/data1/share/kosm/data/ccall_cred_hll_3salts_train.tsv', 'w') as fo:
        with open('/data1/share/kosm/data/ccall_cred_hll_3salts_test.tsv', 'w') as fot:
            for line in f:
                (phone_mobile, approve, call_ymd, urlfr, ymd) = line.strip().split('\t')
                key = '%s_%s_%s' % (phone_mobile, approve, call_ymd)
                if phone_mobile != prev_phone_mobile:
                    if prev_phone_mobile is not None:
                        for key in buf:
                            if key.split('_')[2] < '2016-12-01':
                                fo.write('%s\t%s\n' % (key.split('_')[1], '\t'.join([str(e) for e in buf[key].registers()])))
                            else:
                                fot.write('%s\t%s\n' % (key.split('_')[1], '\t'.join([str(e) for e in buf[key].registers()])))
                    prev_phone_mobile = phone_mobile
                    buf = {}
                    i += 1
                    if (i % 10000 == 0):
                        print(i)
                if(ymd < call_ymd):
                    if not key in buf:
                        buf[key] = Hll(hll_len)
                    for s in salts:
                        buf[key].add(urlfr + s)
            

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000


In [12]:
!shuf /data1/share/kosm/data/ccall_cred_hll_3salts_test.tsv > /data1/share/kosm/data/ccall_cred_hll_test_3salts_shuf.tsv

In [13]:
!shuf /data1/share/kosm/data/ccall_cred_hll_3salts_train.tsv > /data1/share/kosm/data/ccall_cred_hll_train_3salts_shuf.tsv

In [11]:
! wc -l /data1/share/kosm/data/ccall_cred_hll_*_shuf.tsv

          0 /data1/share/kosm/data/ccall_cred_hll_test_3salts_shuf.tsv
      58260 /data1/share/kosm/data/ccall_cred_hll_test_shuf.tsv
          0 /data1/share/kosm/data/ccall_cred_hll_train_3salts_shuf.tsv
     648888 /data1/share/kosm/data/ccall_cred_hll_train_shuf.tsv
     707148 total


In [ ]:
!time /home/a.melnikov/tmp/LightGBM-master/lightgbm config=/home/a.melnikov/projects/web-scoring/train-hll-tri.conf

In [18]:
! cat /home/k.osminin/lgbm_files/train.conf > /home/k.osminin/lgbm_files/cred_salts_log_20170322_2.txt
! nohup /data1/share/LightGBM/lightgbm config=/home/k.osminin/lgbm_files/train.conf data=/data1/share/kosm/data/ccall_cred_hll_train_3salts_shuf.tsv valid=/data1/share/kosm/data/ccall_cred_hll_test_3salts_shuf.tsv >> /home/k.osminin/lgbm_files/cred_salts_log_20170322_2.txt  &2>1

nohup: redirecting stderr to stdout


In [24]:
! tail -2500 /home/k.osminin/lgbm_files/cred_salts_log_20170322_2.txt

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading data in 1437.855388 seconds
[LightGBM] [Info] Number of postive: 238290, number of negative: 410598
[LightGBM] [Info] Number of data: 648888, number of features: 16384
[LightGBM] [Info] Finished initializing training
[LightGBM] [Info] Started training...
[LightGBM] [Info] Iteration: 1, training's : log loss : 0.692014
[LightGBM] [Info] Iteration: 1, training's : AUC : 0.574255
[LightGBM] [Info] Iteration: 1, /data1/share/kosm/data/ccall_cred_hll_test_3salts_shuf.tsv's : log loss : 0.691880
[LightGBM] [Info] Iteration: 1, /data1/share/kosm/data/ccall_cred_hll_test_3salts_shuf.tsv's : AUC : 0.580507
[LightGBM] [Info] 39.784479 seconds elapsed, finished iteration 1
[LightGBM] [Info] Iteration: 2, training's : log loss : 0.690958
[LightGBM] [Info] Iteration: 2, training's : AUC : 0.583702
[LightGBM] [Info] Iteration: 2, /data1/share/kosm/data/ccall_cred_hll_test_3salts_shuf.tsv's : log loss : 0.690699
[LightGB

#### 3 соли. Лучший AUC ROC 0.631. Iteration: 237.